In [1]:
# parse the puzzle meta data into actual map dict
import re
import pickle
from all_puzzle_settings import *

# Function_operator_puzzles

In [2]:
with open('data/Function_operator_puzzle_functions.pkl', 'rb') as file:
    Function_operator_puzzle_functions = pickle.load(file)

In [3]:
def count_unique_integers(expression_list):
    unique_integers = set()
    for expression in expression_list:
        matches = re.findall(r'(?<!\^)-?\b\d+\b(?!/)', expression)
        unique_integers.update(map(int, matches))
    return len(unique_integers), unique_integers
def generate_puzzle_dict_with_meta_function(functions, level):
    if level == 1:
        able_to_see_functions = 'True'
    elif level == 2:
        able_to_see_functions = 'False'
    else:
        assert False, 'Level should be 1 or 2, 1 for oracle setting, 2 for normal setting'
    function_string = ','.join(functions)
    num_unknown_parameters, _ = count_unique_integers(functions)
    description_string = {1:'''You are {Name}, you need to assign values to the functions displayed on the <Computer>, determin the value of 'a' and input it into the <Code secured door> to open it.''',
                          2:'''You are {Name}. You need to assign values to the functions displayed on the <Computer>, determine the values of 'a' and 'b'. Then, input these values into the <Code secured door> in alphabetical order to open it.''',
                          3:'''You are {Name}. You need to assign values to the functions displayed on the <Computer>, determine the values of 'a', 'b', 'c'. Then, input these values into the <Code secured door> in alphabetical order to open it.''',
                          4:'''You are {Name}. You need to assign values to the functions displayed on the <Computer>, determine the values of 'a', 'b', 'c', 'd'. Then, input these values into the <Code secured door> in alphabetical order to open it.'''}

    template_dict = {'Map':{'Puzzle room':[]},
                # item: {Object_id:int, Keyword:str, Information:str, Success_condition:str}  # can pass an dict with these keywords to modify the object with new information and keyword
               'Space_items': {'Puzzle room':[{'Object_id':68,'Special_label':f'''{able_to_see_functions},{function_string}'''},69]},
               'Space_item_containers':{},
               'Edges': {},
               'Agent': {'Name':'Sam', 
                         'Current_situation':description_string[num_unknown_parameters],
                         'Items':{},
                         'Init_position':'Puzzle room',
                         'Solid_Memory':{'Your identity':'You\'re a tourist in new york City.',},# Currently not in use
                         'Action_style':{'Curious':'You develop a keen sense of curiosity about everything.',}, # Currently not in use
                        },
               'Optimal_step_count':50
               }
    return template_dict

In [4]:
all_puzzle_dict = {}
for round_index in range(1,6):
    all_puzzle_dict[f'round{round_index}'] = {}
    all_puzzle_dict[f'round{round_index}']['Level1'] = {}
    all_puzzle_dict[f'round{round_index}']['Level2'] = {}
    for puzzle_index, function in enumerate(Function_operator_puzzle_functions[f'round{round_index}']):
        level1_puzzle_setting_dict = generate_puzzle_dict_with_meta_function(list(function), 1)
        all_puzzle_dict[f'round{round_index}']['Level1'].update({f'puzzle{puzzle_index+1}':level1_puzzle_setting_dict})
        level2_puzzle_setting_dict = generate_puzzle_dict_with_meta_function(list(function), 2)
        all_puzzle_dict[f'round{round_index}']['Level2'].update({f'puzzle{puzzle_index+1}':level2_puzzle_setting_dict})
with open('data/Function_operator_puzzles', 'wb') as file:
    pickle.dump(all_puzzle_dict, file)

In [5]:
#for item in zip(all_puzzle_dict['round1'], Function_operator_puzzles):
different_count = 0
for level in [1,2]:
    for puzzle_index in range(1,21):
        if all_puzzle_dict['round1'][f'Level{level}'][f'puzzle{puzzle_index}'] != Function_operator_puzzles[f'Level{level}'][f'puzzle{puzzle_index}']:
            different_count += 1
            print(f'''Level{level}------------------------------puzzle_index{puzzle_index}------------------------------''')
            print(f'''Level{level}------------------------------puzzle_index{puzzle_index}------------------------------''')
            print(all_puzzle_dict['round1'][f'Level{level}'][f'puzzle{puzzle_index}'])
            print('-----------')
            print(Function_operator_puzzles[f'Level{level}'][f'puzzle{puzzle_index}'])
# only the order of the funcitons is changed due to frozenset the reconstruction is correct

Level1------------------------------puzzle_index7------------------------------
Level1------------------------------puzzle_index7------------------------------
{'Map': {'Puzzle room': []}, 'Space_items': {'Puzzle room': [{'Object_id': 68, 'Special_label': 'True,2*|x|,7*sin(x)'}, 69]}, 'Space_item_containers': {}, 'Edges': {}, 'Agent': {'Name': 'Sam', 'Current_situation': "You are {Name}. You need to assign values to the functions displayed on the <Computer>, determine the values of 'a' and 'b'. Then, input these values into the <Code secured door> in alphabetical order to open it.", 'Items': {}, 'Init_position': 'Puzzle room', 'Solid_Memory': {'Your identity': "You're a tourist in new york City."}, 'Action_style': {'Curious': 'You develop a keen sense of curiosity about everything.'}}, 'Optimal_step_count': 50}
-----------
{'Map': {'Puzzle room': []}, 'Space_items': {'Puzzle room': [{'Object_id': 68, 'Special_label': 'True,7*sin(x),2*|x|'}, 69]}, 'Space_item_containers': {}, 'Edges': {

# Escape room puzzles

In [7]:
with open('data/Escape_room_puzzle_paints.pkl', 'rb') as file:
    Escape_room_puzzle_paints = pickle.load(file)

In [8]:
Paintings_mapping = {
    'oil':{'Blue': [44,49,50,51],
           'Yellow': [45,47],
           'Black': [46],
           'Green': [48]},
    'acrylic': {'Blue':[56,57],
                'Yellow':[53],
                'Black':[52],
                'Green':[54,55]},
    'watercolor': {'Blue': [60,61,62],
                   'Yellow':[58],
                   'Green':[59],
                   'Black':[]}
}
def parse_paints_in_multi_room(paints, level):
    item_dict = {'Oil Painting Gallery':[{'Object_id':65,'Special_label':f'Level{level}'}],
                 'Acrylic Painting Gallery':[],
                 'Watercolour Gallery':[]}
    paint_room_mapping = {'oil':[],
                          'acrylic':[],
                          'watercolor':[],}
    for item in Paintings_mapping['oil'].items():
        paint_room_mapping['oil'].extend(item[1])

    for item in Paintings_mapping['acrylic'].items():
        paint_room_mapping['acrylic'].extend(item[1])
        
    for item in Paintings_mapping['watercolor'].items():
        paint_room_mapping['watercolor'].extend(item[1])
    
    for paint in paints:
        if paint in paint_room_mapping['oil']:
            item_dict['Oil Painting Gallery'].append(paint)
        elif paint in paint_room_mapping['acrylic']:
            item_dict['Acrylic Painting Gallery'].append(paint)
        elif paint in paint_room_mapping['watercolor']:
            item_dict['Watercolour Gallery'].append(paint)
        else:
            assert False, f'''{paint} is not known key for paints.'''
    return item_dict
        
    
def generate_puzzle_dict_with_meta_function(paints, level, multi_room):
    if not multi_room:
        item_dict = {'Corridor':[{'Object_id':65,'Special_label':f'Level{level}'}]}
        item_dict['Corridor'].extend(paints)
        puzzle_dict =  {'Map':{'Corridor':[]},
                               'Space_items': item_dict,
                               'Space_item_containers':{},
                               'Edges': {},
                               'Agent': {'Name':'Sam',
                                         'Current_situation':'You are {Name} and you are now playing an escape room game where you need to escape this place.', # hard coded in the run_experiment.py
                                         'Items':{},
                                         'Init_position':'Corridor',
                                         'Solid_Memory':{'Your identity':'You\'re a tourist in new york City.',},# Currently not in use
                                         'Action_style':{'Curious':'You develop a keen sense of curiosity about everything.',}, # Currently not in use
                                        },
                               'Optimal_step_count':50, # perceptual_action: 14， try_action:2+3edge
                               }
    else:
        item_dict = parse_paints_in_multi_room(paints, level)
        puzzle_dict ={'Map':{
                          'Oil Painting Gallery':['Watercolour Gallery','Acrylic Painting Gallery'],
                          'Acrylic Painting Gallery':['Watercolour Gallery'],
                          'Watercolour Gallery':['Acrylic Painting Gallery']},
                       'Space_items': item_dict,
                       'Space_item_containers':{},
                       'Edges': {'Oil Painting Gallery':{'Watercolour Gallery':[31], 'Acrylic Painting Gallery':[30]}},
                       'Agent': {'Name':'Sam',
                                 'Current_situation':'You are {Name} and you are now playing an escape room game where you need to escape this place.',
                                 'Items':{},
                                 'Init_position':'Oil Painting Gallery',
                                 'Solid_Memory':{'Your identity':'You\'re a tourist in new york City.',},# Currently not in use
                                 'Action_style':{'Curious':'You develop a keen sense of curiosity about everything.',}, # Currently not in use
                                },
                       'Optimal_step_count':50 # perceptual_action: 11, # try action: 1+3
                        }
    return puzzle_dict
all_puzzle_dict = {}  
for round_index in range(1,6):
    all_puzzle_dict[f'round{round_index}'] = {}
    all_puzzle_dict[f'round{round_index}']['Level1'] = {}
    all_puzzle_dict[f'round{round_index}']['Level2'] = {}
    for puzzle_index, paint_combination in enumerate(Escape_room_puzzle_paints[f'round{round_index}']):
        paint_combinational = list(paint_combination)
        level_1_single = generate_puzzle_dict_with_meta_function(paint_combinational, 1, False)
        level_1_multi = generate_puzzle_dict_with_meta_function(paint_combinational, 1, True)
        all_puzzle_dict[f'round{round_index}']['Level1'].update({f'puzzle{puzzle_index+1}':level_1_single})
        all_puzzle_dict[f'round{round_index}']['Level1'].update({f'puzzle{puzzle_index+11}':level_1_multi})
        
        level_2_single = generate_puzzle_dict_with_meta_function(paint_combinational, 2, False)
        level_2_multi = generate_puzzle_dict_with_meta_function(paint_combinational, 2, True)
        all_puzzle_dict[f'round{round_index}']['Level2'].update({f'puzzle{puzzle_index+1}':level_2_single})
        all_puzzle_dict[f'round{round_index}']['Level2'].update({f'puzzle{puzzle_index+11}':level_2_multi})
        


In [9]:
#for item in zip(all_puzzle_dict['round1'], Function_operator_puzzles):
different_count = 0
for level in [1,2]:
    for puzzle_index in range(1,21):
        puzzle_reconstruct = all_puzzle_dict['round1'][f'Level{level}'][f'puzzle{puzzle_index}']
        puzzle_hand_made = Art_gallery_puzzles[f'Level{level}'][f'puzzle{puzzle_index}']
        if set(puzzle_reconstruct['Space_items']) != set(puzzle_hand_made['Space_items']):
            different_count+=1
            print(puzzle_reconstruct['Space_items'], puzzle_hand_made['Space_items'])
        else:
            print(puzzle_reconstruct['Space_items'], puzzle_hand_made['Space_items'])
        puzzle_reconstruct['Space_items'] = []
        puzzle_hand_made['Space_items'] = []
        if puzzle_reconstruct != puzzle_hand_made:
            different_count += 1
            print(f'''Level{level}------------------------------puzzle_index{puzzle_index}------------------------------''')
            print(f'''Level{level}------------------------------puzzle_index{puzzle_index}------------------------------''')
            print(all_puzzle_dict['round1'][f'Level{level}'][f'puzzle{puzzle_index}'])
            print('-----------')
            print(Art_gallery_puzzles[f'Level{level}'][f'puzzle{puzzle_index}'])
# only the order of the paintings is changed due to frozenset the reconstruction is correct, some minor differences in cur-situation is handled in the run_experiments

{'Corridor': [{'Object_id': 65, 'Special_label': 'Level1'}, 56, 50, 59]} {'Corridor': [{'Object_id': 65, 'Special_label': 'Level1'}, 50, 56, 59]}
Level1------------------------------puzzle_index1------------------------------
Level1------------------------------puzzle_index1------------------------------
{'Map': {'Corridor': []}, 'Space_items': [], 'Space_item_containers': {}, 'Edges': {}, 'Agent': {'Name': 'Sam', 'Current_situation': 'You are {Name} and you are now playing an escape room game where you need to escape this place.', 'Items': {}, 'Init_position': 'Corridor', 'Solid_Memory': {'Your identity': "You're a tourist in new york City."}, 'Action_style': {'Curious': 'You develop a keen sense of curiosity about everything.'}}, 'Optimal_step_count': 50}
-----------
{'Map': {'Corridor': []}, 'Space_items': [], 'Space_item_containers': {}, 'Edges': {}, 'Agent': {'Name': 'Sam', 'Current_situation': 'You are {Name} ', 'Items': {}, 'Init_position': 'Corridor', 'Solid_Memory': {'Your ide

In [10]:
different_count

3

In [11]:
with open('data/Art_gallery_puzzles','wb') as file:
    pickle.dump(all_puzzle_dict, file)

# Reactor puzzles

In [12]:
with open('data/Reactor_puzzle_strings.pkl', 'rb') as file:
    Reactor_puzzle_strings = pickle.load(file)
    
Reactor_puzzle_strings

{'round1': ['ACB', 'CCADD', 'CADEA', 'FEADE', 'ABCDEF'],
 'round2': ['FEC', 'AFEBC', 'FFAAF', 'BDCDD', 'BACAAC'],
 'round3': ['EEA', 'EAEFA', 'DCBAC', 'CCECC', 'CFDEDB'],
 'round4': ['CFC', 'DCADF', 'EDEDF', 'AFEBE', 'DEBCED'],
 'round5': ['DBB', 'BCCBB', 'BDCAC', 'DEEDF', 'CBAFDE']}

In [13]:
letters = ['A','B','C','D','E','F']
def find_the_last_letter(target_string):

    cur_index = 0
    for letter in target_string:
        new_index = letters.index(letter)
        if new_index > cur_index:
            cur_index = new_index
    return cur_index+1

def construct_dict(target_string, rule_index,):
    assert rule_index in [1,2,3,4],f'{rule_index} is not an known rule.'
    max_letter_index = find_the_last_letter(target_string)
    init_letter_string = ','.join(letters[:max_letter_index])
    puzzle_dict = {'Map':{'Chemical laboratory':[]},
                    # item: {Object_id:int, Keyword:str, Information:str, Success_condition:str}  # can pass an dict with these keywords to modify the object with new information and keyword
                   'Space_items': {'Chemical laboratory':[{'Object_id':66,'Information':'''It's a very advanced material reactor that can put up to two of any material into it to create a reaction. The reactor provides unlimited one-letter raw material and other materials you've already synthesized.''','Special_label':f'{rule_index},{init_letter_string}'},{'Object_id':64,'Special_label':target_string}]},
                   'Space_item_containers':{},
                   'Edges': {},
                   'Agent': {'Name':'Sam',
                             'Current_situation':'''You are {Name}. You need to research and generate the corresponding chemical material required in the <Task Monitor>''',
                             'Items':{},
                             'Init_position':'Chemical laboratory',
                             'Solid_Memory':{'Your identity':'You\'re a tourist in new york City.',},# Currently not in use
                             'Action_style':{'Curious':'You develop a keen sense of curiosity about everything.',}, # Currently not in use
                            },
                   'Optimal_step_count':50 # perceptual_action: 4, # try action: pick 3A 1B 1C react A+A, AA+A, AAA+BC, B+C (9 actions), put_in, A,A, AA,A, AAA, B,C, BC (8 Actions)
                   }
    return puzzle_dict
    
all_puzzle_dict = {}
for round_index in range(1,6):
    all_puzzle_dict[f'round{round_index}'] = {}
    all_puzzle_dict[f'round{round_index}']['Level1'] = {}
    all_puzzle_dict[f'round{round_index}']['Level2'] = {}
    for puzzle_index, target_string in enumerate(Reactor_puzzle_strings[f'round{round_index}']):
        for rule_index in [1,2,3,4]:
            level1_dict = construct_dict(target_string, rule_index)
            all_puzzle_dict[f'round{round_index}']['Level1'].update({f'puzzle{puzzle_index+1+(rule_index-1)*5}':level1_dict})
            level2_dict = construct_dict(target_string, rule_index) # the rule for oracle are provided in the run_experiment.py the cur_situation is overwritten there
            all_puzzle_dict[f'round{round_index}']['Level2'].update({f'puzzle{puzzle_index+1+(rule_index-1)*5}':level2_dict})
        

In [14]:
different_count = 0
for level in [1,2]:
    for puzzle_index in range(1,21):
        puzzle_reconstruct = all_puzzle_dict['round1'][f'Level{level}'][f'puzzle{puzzle_index}']
        puzzle_hand_made = Reactor_puzzles[f'Level{level}'][f'puzzle{puzzle_index}']
        if puzzle_reconstruct != puzzle_hand_made:
            different_count += 1
            print(f'''Level{level}------------------------------puzzle_index{puzzle_index}------------------------------''')
            print(f'''Level{level}------------------------------puzzle_index{puzzle_index}------------------------------''')
            print(all_puzzle_dict['round1'][f'Level{level}'][f'puzzle{puzzle_index}'])
            print('-----------')
            print(Reactor_puzzles[f'Level{level}'][f'puzzle{puzzle_index}'])
# only one puzzle changed because switched the order of that puzzle in ABCDEF and FEADE reconstruction success

Level1------------------------------puzzle_index4------------------------------
Level1------------------------------puzzle_index4------------------------------
{'Map': {'Chemical laboratory': []}, 'Space_items': {'Chemical laboratory': [{'Object_id': 66, 'Information': "It's a very advanced material reactor that can put up to two of any material into it to create a reaction. The reactor provides unlimited one-letter raw material and other materials you've already synthesized.", 'Special_label': '1,A,B,C,D,E,F'}, {'Object_id': 64, 'Special_label': 'FEADE'}]}, 'Space_item_containers': {}, 'Edges': {}, 'Agent': {'Name': 'Sam', 'Current_situation': 'You are {Name}. You need to research and generate the corresponding chemical material required in the <Task Monitor>', 'Items': {}, 'Init_position': 'Chemical laboratory', 'Solid_Memory': {'Your identity': "You're a tourist in new york City."}, 'Action_style': {'Curious': 'You develop a keen sense of curiosity about everything.'}}, 'Optimal_ste

In [3]:
import pickle
with open('../../data/Function_operator_puzzles', 'rb') as file:
    Function_operator_puzzles_enhanced = pickle.load(file)

In [5]:
Function_operator_puzzles_enhanced['round1']

{'Level1': {'puzzle1': {'Map': {'Puzzle room': []},
   'Space_items': {'Puzzle room': [{'Object_id': 68,
      'Special_label': 'True,6'},
     69]},
   'Space_item_containers': {},
   'Edges': {},
   'Agent': {'Name': 'Sam',
    'Current_situation': "You are {Name}, you need to assign values to the functions displayed on the <Computer>, determin the value of 'a' and input it into the <Code secured door> to open it.",
    'Items': {},
    'Init_position': 'Puzzle room',
    'Solid_Memory': {'Your identity': "You're a tourist in new york City."},
    'Action_style': {'Curious': 'You develop a keen sense of curiosity about everything.'}},
   'Optimal_step_count': 50},
  'puzzle2': {'Map': {'Puzzle room': []},
   'Space_items': {'Puzzle room': [{'Object_id': 68,
      'Special_label': 'True,1*x'},
     69]},
   'Space_item_containers': {},
   'Edges': {},
   'Agent': {'Name': 'Sam',
    'Current_situation': "You are {Name}, you need to assign values to the functions displayed on the <Comp

In [15]:
with open('data/Reactor_puzzles', 'wb') as file:
    pickle.dump(all_puzzle_dict, file)